In [2]:
import findspark

findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [6]:
spark = (SparkSession.
    builder.appName("spark project").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/19 05:29:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
book = spark.read.text("./1342-0.txt")

In [10]:
book.show(5, truncate = 30)

+------------------------------+
|                         value|
+------------------------------+
|The Project Gutenberg EBook...|
|                              |
|This eBook is for the use o...|
|almost no restrictions what...|
|re-use it under the terms o...|
+------------------------------+
only showing top 5 rows



In [33]:
from pyspark.sql.functions import col, explode, split, regexp_extract, lower

In [27]:
make_split = book.select(split(col("value"), " ").alias("lower_col"))

In [28]:
make_split.show(5)

+--------------------+
|           lower_col|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows



In [29]:
make_explode = make_split.select(explode(col("lower_col")).alias("explode_col"))

In [30]:
make_explode.show(5)

+-----------+
|explode_col|
+-----------+
|        The|
|    Project|
|  Gutenberg|
|      EBook|
|         of|
+-----------+
only showing top 5 rows



In [35]:
make_lower = make_explode.select(lower(col("explode_col")).alias("lower_col"))

In [37]:
make_lower.show(15)

+----------+
| lower_col|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
+----------+
only showing top 15 rows



In [38]:
make_clean = make_lower.select(regexp_extract(col("lower_col"), "[a-z]+", 0).alias("clean_col"))

In [39]:
make_clean.show(15)

+---------+
|clean_col|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|         |
|     this|
|    ebook|
|       is|
+---------+
only showing top 15 rows



In [40]:
removed_spaces = make_clean.filter(col("clean_col") != "")

In [41]:
removed_spaces.show(15)

+---------+
|clean_col|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|     this|
|    ebook|
|       is|
|      for|
+---------+
only showing top 15 rows

